## What an LLM is (in practice)

A **Large Language Model (LLM)** is a neural network trained to predict the next token (piece of text) given prior tokens. That simple objective turns into useful capabilities—summarization, extraction, Q&A, code generation—because the model has learned statistical structure from massive text/code corpora.

In an app development, you usually interact with an LLM in one of two ways:

* **Chat mode**: a sequence of `{role, content}` messages (system/developer/user/assistant).
* **Completion mode**: one big prompt string (less common now, but still used).


## Local LLMs: what changes vs “cloud LLMs”

A **local LLM** means the model weights run on *your* hardware (laptop/desktop/server), so:

**Pros**

* **Cost control**: no per-token API bill; you mainly “pay” in hardware + electricity.
* **Privacy & data locality**: sensitive docs never have to leave your machine.
* **Offline / air-gapped**: possible for secure environments. ([Ollama][1])

**Cons**

* **Throughput + latency** depends on your CPU/GPU and quantization choices.
* **Model size constraints**: big models can be impractical without a strong GPU / lots of RAM/VRAM.
* **Ops overhead**: downloads, storage, model management, and performance tuning.

Here is an article I recommend on this: [You're using your local LLM wrong if you're prompting it like a cloud LLM](https://www.xda-developers.com/youre-using-local-llm-wrong-if-youre-prompting-it-like-cloud-llm/)

## Ollama

**Ollama** is a tool that lets you run Large Language Models directly on your own computer, no cloud required. No subscriptions. No sending your data to Big Tech's servers. It makes local models easy to:

* download/pull and run from CLI
* serve over a **local HTTP API** (default `http://localhost:11434/api`) ([Ollama Documentation][2])
* integrate into apps with official client libraries (Python/JS) ([Ollama Documentation][2])

It also supports GPU acceleration across common stacks (NVIDIA CUDA, AMD ROCm, Apple Metal, and experimental Vulkan paths). ([Ollama Documentation][3])


## Costs: what you actually pay for

### 1) Ollama software cost

Ollama has a **Free** plan for running models on your own hardware; paid tiers mainly relate to *cloud* usage and extras. ([Ollama][1])

### 2) Local inference cost drivers (the real ones)

* **Hardware**: GPU/VRAM (or CPU/RAM) is the main limiter.
* **Electricity/thermals**: sustained inference draws power; laptops may throttle.
* **Time cost**: slower tokens/sec can be more expensive than cloud for “human time.”

### 3) Why quantization matters (and why everyone talks about it)

Local setups often rely on **quantized** weights (e.g., 4–8 bit) to reduce memory and run bigger models. A recent empirical study focusing on llama.cpp quantization highlights that different quantization schemes trade off **quality vs speed vs memory** and should be chosen based on your hardware and task needs. ([arXiv][4])

## Resources you need

Think in three knobs:

1. **Model size** (e.g., 3B, 8B, 14B parameters)
2. **Quantization level** (e.g., 4-bit vs 8-bit)
3. **Context length** (how many tokens you keep “in memory”)

**Context length is a hidden VRAM tax.** Ollama even defaults context length based on VRAM tiers (e.g., <24 GiB → 4k context; 24–48 GiB → 32k; ≥48 GiB → 256k). Larger context increases memory requirements. ([Ollama Documentation][5])

## Prompt techniques that matter *more* for local/smaller LLMs

Smaller local models are less “instruction-sticky” than frontier cloud models, so you get better results by being **more structured** and **more constrained**.

### 1) Use *tight* instructions + explicit output schemas

Local models drift more. Give:

* a short role (“You are an information extraction engine.”)
* constraints (“Return valid JSON only.”)
* a schema example (keys, types)
* a **single** success criterion (what “good” looks like)

### 2) Few-shot, but *small* and highly relevant

Few-shot prompting (2–5 examples) often boosts reliability because you’re showing the exact pattern you want. Keep examples short and format-identical to the target output. ([promptingguide.ai][6])

### 3) Delimiters and sectioning (reduce confusion)

Use clear separators like:

* `### Instruction`
* `### Input`
* `### Output (JSON)`

This helps weaker models not mix instructions with data.

### 4) Prefer lower temperature + controlled decoding for consistency

For extraction/summarization/metadata tasks:

* temperature: **0.0–0.3** (start low)
* if outputs repeat or ramble, reduce randomness further and add stricter formatting constraints

Also: if you’re using chat models via Hugging Face tooling, **chat template / prompt wrapping** mistakes can badly degrade controllability—this bites local setups a lot. ([Hugging Face Forums][7])

### 5) Keep context short; “stuffing” harms smaller models faster

Instead of pasting huge documents:

* chunk text
* summarize per chunk
* then merge summaries
  This is usually better than trying to brute-force long context on a small model (and cheaper in VRAM). ([Ollama Documentation][5])

### 6) Use “format-first” prompting for structured tasks

A reliable pattern for local LLMs:

1. Provide JSON schema
2. Provide 1 example input/output
3. Provide the real input
4. Demand “JSON only” (no prose)

This reduces “helpful chatter” and failure modes.

# **Useful Ollama CLI commands**

### Help / version

* `ollama -h` / `ollama --help` — show commands and flags
* `ollama --version` — show installed version

### Run & serve (inference)

* `ollama run <model>` — run a model interactively (chat in terminal)

  * Example: `ollama run qwen2.5:3b`
* `ollama serve` (alias: `start`) — start the local Ollama server (HTTP API)
* `ollama stop <model>` — stop a running model
* `ollama ps` — list running models (what’s loaded / active)

### Model management

* `ollama pull <model>` — download a model from a registry 
* `ollama list` (often `ollama ls`) — list models you have downloaded 
* `ollama show <model>` — show model info/config (params, context length, etc.) 
* `ollama rm <model>` — remove a local model 
* `ollama cp <src> <dst>` — copy/duplicate a model locally (handy for variants) 
* `ollama push <model>` — push a model to a registry (requires account/auth) 

### Custom models / modelfiles

* `ollama create <new_model> -f <Modelfile>` — create a custom model definition (system prompt, params, base model, etc.) 

### Common “day-1” workflow

```bash
ollama pull qwen2.5:3b
ollama run qwen2.5:3b
ollama serve
ollama ps
ollama stop qwen2.5:3b
```
### References and useful links
- [CLI Reference](https://docs.ollama.com/cli)  
- [Ollama example](https://docs.radxa.com/en/cubie/a7s/app-dev/ollama-dev/ollama-example) 
- [Ollama CLI tutorial: Learn to use Ollama in the terminal](https://www.hostinger.com/tutorials/ollama-cli-tutorial) 


> Content created by [**Carlos Cruz-Maldonado**](https://www.linkedin.com/in/carloscruzmaldonado/).  
> I am available to answer any questions or provide further assistance.   
> Feel free to reach out to me at any time.